In [1]:
import sys
from pathlib import Path
ROOT_DIR = Path().resolve().parents[0]
sys.path.append(str(ROOT_DIR))
import config as cfg

from transformers import (
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback
)
import numpy as np
from sklearn.metrics import roc_auc_score
from datasets import load_from_disk
import os
import json

N_RUN = 3               # Number of run to separe different experiments

In [2]:
def get_fold_datasets(ds, fold):
    ds_train = ds.filter(lambda x: x["fold"] != fold)
    ds_val = ds.filter(lambda x: x["fold"] == fold)
    return ds_train, ds_val

In [3]:
def model_init(model_name=cfg.MODEL_BASE):
    return AutoModelForSequenceClassification.from_pretrained(
        model_name,
        num_labels=6,
        problem_type="multi_label_classification",
    )

In [4]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    probs = 1 / (1 + np.exp(-logits))
    auc = roc_auc_score(labels, probs, average="macro")
    return {"roc_auc_macro": auc}

In [5]:
def print_highlighted_box(text, wide_chars):
    spaces = (wide_chars - len(text)) // 2
    left_spaces = spaces - 1
    right_spaces = spaces if (wide_chars - len(text)) % 2 else spaces - 1
    print(f"{'-' * wide_chars}")
    print(f"|{' ' * (wide_chars - 2)}|")
    print(f"|{' ' * left_spaces}{text}{' ' * right_spaces}|")
    print(f"|{' ' * (wide_chars - 2)}|")
    print(f"{'-' * wide_chars}")

In [6]:
class TrainerWithTrainMetrics(Trainer):

    def evaluate(
        self,
        eval_dataset=None,
        ignore_keys=None,
        metric_key_prefix: str = "eval",
    ):
		# Validation metrics (what is usually returned by Trainer.evaluate)
        metrics = super().evaluate(
            eval_dataset=eval_dataset,
            ignore_keys=ignore_keys,
            metric_key_prefix=metric_key_prefix,
        )

        # Training metrics (added in this custom Trainer)
        train_metrics = super().evaluate(
            eval_dataset=self.train_dataset,
            ignore_keys=ignore_keys,
            metric_key_prefix="train",
        )

        # Combine metrics
        metrics.update(train_metrics)
        return metrics

In [7]:
ds_train_tokenized = load_from_disk(cfg.PATH_DS_TRAIN_TOKENIZED)
ds_train_tokenized

Dataset({
    features: ['id', 'fold', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 159571
})

In [8]:
for fold in range(cfg.N_FOLDS):

    path_checkpoint_dir = os.path.join(cfg.PATH_CHECKPOINTS, cfg.MODEL_BASE, f"run_{N_RUN}", f"fold_{fold}")
    path_model_final = os.path.join(path_checkpoint_dir, "model_final")

    ds_train, ds_val = get_fold_datasets(ds_train_tokenized, fold)
    print_highlighted_box(f"FOLD {fold}: TRAIN SIZE: {len(ds_train)} ({len(ds_train)/len(ds_train_tokenized):.2%}), VAL SIZE: {len(ds_val)} ({len(ds_val)/len(ds_train_tokenized):.2%})", 80)

    args = TrainingArguments(
        # Training structure parameters
        num_train_epochs=cfg.EPOCHS,
        per_device_train_batch_size=cfg.BATCH_SIZE,
        gradient_accumulation_steps=1,
        per_device_eval_batch_size=cfg.BATCH_SIZE,
        # Optimization parameters
        learning_rate=cfg.LEARNING_RATE,
        weight_decay=0.01,
        optim="adamw_torch_fused",
        # Evaluation and saving parameters
        eval_strategy="steps",
        save_strategy="steps",
        eval_steps=cfg.EVAL_STEPS,
        save_steps=cfg.SAVE_STEPS,
        load_best_model_at_end=True,
        save_only_model=True,
        save_total_limit=cfg.SAVE_TOTAL_LIMIT,
        metric_for_best_model="eval_roc_auc_macro",
        # Precision and memory parameters
        fp16=True,
        gradient_checkpointing=False,
        dataloader_num_workers=2,
        dataloader_pin_memory=True,
        # Logging and reproducibility parameters
        logging_steps=cfg.LOGGING_STEPS,
        seed=cfg.RANDOM_SEED,
        output_dir= path_checkpoint_dir,
    )

    trainer = TrainerWithTrainMetrics(
        model=model_init(cfg.MODEL_BASE),
		args=args,
		train_dataset=ds_train,
		eval_dataset=ds_val,
		compute_metrics=compute_metrics,
		callbacks=[EarlyStoppingCallback(early_stopping_patience=cfg.EARLY_STOP_PATIENCE)]
	)

    train_results = trainer.train()
    
	# Save log_history of the last training to easily access it later
    path_hist = os.path.join(path_checkpoint_dir, "log_history.json")
    with open(path_hist, "w") as f:
        json.dump(trainer.state.log_history, f, indent=2)

	# Save the final model
    trainer.save_model(path_model_final)

--------------------------------------------------------------------------------
|                                                                              |
|        FOLD 0: TRAIN SIZE: 127656 (80.00%), VAL SIZE: 31915 (20.00%)         |
|                                                                              |
--------------------------------------------------------------------------------


Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss,Validation Loss,Roc Auc Macro
250,0.073200,0.060197,0.942345
500,0.065200,0.054832,0.950260
750,0.048900,0.047950,0.976738
1000,0.048900,0.045996,0.978491
1250,0.041500,0.043530,0.978753
1500,0.038200,0.044416,0.979025
1750,0.039200,0.043646,0.980857
2000,0.043100,0.040924,0.980842
2250,0.046900,0.042129,0.981409
2500,0.063800,0.051611,0.983353


early stopping required metric_for_best_model, but did not find eval_roc_auc_macro so early stopping is disabled
early stopping required metric_for_best_model, but did not find eval_roc_auc_macro so early stopping is disabled
early stopping required metric_for_best_model, but did not find eval_roc_auc_macro so early stopping is disabled
early stopping required metric_for_best_model, but did not find eval_roc_auc_macro so early stopping is disabled
early stopping required metric_for_best_model, but did not find eval_roc_auc_macro so early stopping is disabled
early stopping required metric_for_best_model, but did not find eval_roc_auc_macro so early stopping is disabled
early stopping required metric_for_best_model, but did not find eval_roc_auc_macro so early stopping is disabled
early stopping required metric_for_best_model, but did not find eval_roc_auc_macro so early stopping is disabled
early stopping required metric_for_best_model, but did not find eval_roc_auc_macro so early stop

--------------------------------------------------------------------------------
|                                                                              |
|        FOLD 1: TRAIN SIZE: 127656 (80.00%), VAL SIZE: 31915 (20.00%)         |
|                                                                              |
--------------------------------------------------------------------------------


...

early stopping required metric_for_best_model, but did not find eval_roc_auc_macro so early stopping is disabled
early stopping required metric_for_best_model, but did not find eval_roc_auc_macro so early stopping is disabled
early stopping required metric_for_best_model, but did not find eval_roc_auc_macro so early stopping is disabled
early stopping required metric_for_best_model, but did not find eval_roc_auc_macro so early stopping is disabled
early stopping required metric_for_best_model, but did not find eval_roc_auc_macro so early stopping is disabled
early stopping required metric_for_best_model, but did not find eval_roc_auc_macro so early stopping is disabled
early stopping required metric_for_best_model, but did not find eval_roc_auc_macro so early stopping is disabled
early stopping required metric_for_best_model, but did not find eval_roc_auc_macro so early stopping is disabled
early stopping required metric_for_best_model, but did not find eval_roc_auc_macro so early stop

--------------------------------------------------------------------------------
|                                                                              |
|        FOLD 4: TRAIN SIZE: 127656 (80.00%), VAL SIZE: 31915 (20.00%)         |
|                                                                              |
--------------------------------------------------------------------------------


Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss,Validation Loss,Roc Auc Macro
250,0.048500,0.058558,0.949990
500,0.059500,0.048698,0.974947
750,0.050400,0.049351,0.974917
1000,0.039100,0.045135,0.977030
1250,0.034600,0.047797,0.964732
1500,0.042400,0.044076,0.980219
1750,0.057700,0.041564,0.981511
2000,0.037800,0.041858,0.981498
2250,0.040100,0.043832,0.982573
2500,0.044100,0.038637,0.983151


early stopping required metric_for_best_model, but did not find eval_roc_auc_macro so early stopping is disabled
early stopping required metric_for_best_model, but did not find eval_roc_auc_macro so early stopping is disabled
early stopping required metric_for_best_model, but did not find eval_roc_auc_macro so early stopping is disabled
early stopping required metric_for_best_model, but did not find eval_roc_auc_macro so early stopping is disabled
early stopping required metric_for_best_model, but did not find eval_roc_auc_macro so early stopping is disabled
early stopping required metric_for_best_model, but did not find eval_roc_auc_macro so early stopping is disabled
early stopping required metric_for_best_model, but did not find eval_roc_auc_macro so early stopping is disabled
early stopping required metric_for_best_model, but did not find eval_roc_auc_macro so early stopping is disabled
early stopping required metric_for_best_model, but did not find eval_roc_auc_macro so early stop